In [1]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 

D:\Anaconda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
D:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)

Bad key "text.kerning_factor" on line 4 in
D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import scipy.linalg as slin
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import re
import math
from torch.optim.adam import Adam
from utils import *
from statistics import mean


In [3]:
def simulate_lsem_dynamic(W_all,Z: nx.DiGraph,
                 n: int,n_time:int, treatment_type: str,
                 noise_scale: float = 0.5,
                 baseline: float = 1.0) -> np.ndarray:
    """Simulate samples from LSEM.
        
        Args:
        W_all,A: weigthed DAG for instaneous relation and lagged relation
        n: number of samples in each time-stamp
        lag: degree of AR
        n_time: number of time stamp
        treatment_type: the type of the exposure {Binary, Gaussian}
        noise_scale: noise scale parameter of Gaussian distribution in the lSEM
        baseline: the baseline for the outcome
        
        Returns:
        X: [time,n, d] sample matrix
        """
    #W_array = nx.to_numpy_array(W)
    Z_array = nx.to_numpy_array(Z)
    d = Z_array.shape[0]
    #X_all = np.zeros([n_time+1,n, d])
    
    ## create the initial data
    X = np.zeros([n, d])
    W_0=W_all[0,:,:]
    ordered_vertices = list(nx.topological_sort(nx.from_numpy_matrix(W_0,create_using=nx.DiGraph)))
    assert len(ordered_vertices) == d
    rank_A = ordered_vertices.index(0)
    for j in ordered_vertices:
        if ordered_vertices.index(j) > rank_A:
            parents = list(nx.from_numpy_matrix(W_0,create_using=nx.DiGraph).predecessors(j))
            X[:, j] = X[:, parents].dot(W_0[parents, j]) + np.random.normal(scale=noise_scale, size=n)
        elif ordered_vertices.index(j) < rank_A:
            X[:, j] = np.random.normal(scale=noise_scale, size=n)
        else:
            if treatment_type == 'Binary':
                X[:, j] = 2 * (np.random.binomial(1, 0.5, n) - 0.5)
            elif treatment_type == 'Gaussian':
                X[:, j] = np.random.normal(scale=noise_scale, size=n)
            else:
                raise ValueError('unknown exposure type')
    X[:, d-1] += baseline
    X_all=X
    ## for follow-up time-stamps, X=XW+AZ+E
    for time in range(1,n_time+1):
        X_temp = np.matmul(X,Z_array)
        W_array=W_all[time,:,:] ## different index!
        W=nx.from_numpy_matrix(W_array,create_using=nx.DiGraph)
        for j in ordered_vertices:
            if ordered_vertices.index(j) > rank_A:
                parents = list(W.predecessors(j))
                X_temp[:, j] += X_temp[:, parents].dot(W_array[parents, j]) + np.random.normal(scale=noise_scale, size=n)
            elif ordered_vertices.index(j) < rank_A:
                X_temp[:, j] += np.random.normal(scale=noise_scale, size=n)
            else:
                if treatment_type == 'Binary':
                    X_temp[:, j] += 2 * (np.random.binomial(1, 0.5, n) - 0.5)
                elif treatment_type == 'Gaussian':
                    X_temp[:, j] += np.random.normal(scale=noise_scale, size=n)
                else:
                    raise ValueError('unknown exposure type')
        X_all=np.append(X_all,X_temp,axis=0)
        X=X_temp
    return X_all



In [5]:
import math 
def cos(x):
    return(math.cos(x/4*math.pi)*0.8)

In [6]:
W_all=np.zeros([11,5, 5])
for i in range(11):
    W_all[i,0,4]=cos(i)

In [7]:
## create Z matrix
Z=np.identity(5)
Z[0,0]=0 # no correlation for treatment
Z_graph=nx.from_numpy_matrix(Z,create_using=nx.DiGraph)

## piecewise ANOCA

In [8]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import pickle
import os
import random

import torch.optim as optim
from torch.optim import lr_scheduler
import math
from utils import *

from multiprocessing import Pool
import multiprocessing
n_cores = multiprocessing.cpu_count()
from numpy.random import randn
from random import seed as rseed
from numpy.random import seed as npseed


In [9]:
# compute constraint h(A) value
def _h_A(A, m):
    expm_A = matrix_poly(A*A, m)
    h_A = torch.trace(expm_A) - m
    return h_A

prox_plus = torch.nn.Threshold(0.,0.)

def stau(w, tau):
    w1 = prox_plus(torch.abs(w)-tau)
    return torch.sign(w)*w1


def update_optimizer(optimizer, original_lr, c_A):
    '''related LR to c_A, whenever c_A gets big, reduce LR proportionally'''
    MAX_LR = 1e-2
    MIN_LR = 1e-4

    estimated_lr = original_lr / (math.log10(c_A) + 1e-10)
    if estimated_lr > MAX_LR:
        lr = MAX_LR
    elif estimated_lr < MIN_LR:
        lr = MIN_LR
    else:
        lr = estimated_lr

    # set LR
    for parame_group in optimizer.param_groups:
        parame_group['lr'] = lr

    return optimizer, lr

#===================================
# training:
#===================================

def train(epoch, lambda_A, c_A, optimizer,old_lr):
    t = time.time()
    nll_train = []
    kl_train = []
    mse_train = []
    shd_trian = []

    encoder.train()
    decoder.train()
    scheduler.step()


    # update optimizer
    optimizer, lr = update_optimizer(optimizer, old_lr, c_A)


    for batch_idx, (data, relations) in enumerate(train_loader):

#         if args.cuda:
#             data, relations = data.cuda(), relations.cuda()
        data, relations = Variable(data).double(), Variable(relations).double()

        # reshape data
        relations = relations.unsqueeze(2)

        optimizer.zero_grad()

        enc_x, logits, origin_A, adj_A_tilt_encoder, z_gap, z_positive, myA, Wa = encoder(data, rel_rec, rel_send)  # logits is of size: [num_sims, z_dims]
        edges = logits

        dec_x, output, adj_A_tilt_decoder = decoder(data, edges,d * x_dims, rel_rec, rel_send, origin_A, adj_A_tilt_encoder, Wa)

        if torch.sum(output != output):
            print('nan error\n')

        target = data
        preds = output
        variance = 0.

        # reconstruction accuracy loss
        loss_nll = nll_gaussian(preds, target, variance)

        # KL loss
        loss_kl = kl_gaussian(logits)

        # ELBO loss:
        loss = loss_kl + loss_nll

        # add A loss
        one_adj_A = origin_A # torch.mean(adj_A_tilt_decoder, dim =0)
        sparse_loss = tau_A * torch.sum(torch.abs(one_adj_A))

        # compute h(A)
        h_A = _h_A(origin_A, d)
        loss += lambda_A * h_A + 0.5 * c_A * h_A * h_A + 100. * torch.trace(origin_A*origin_A) + sparse_loss #+  0.01 * torch.sum(variance * variance)


        loss.backward()
        loss = optimizer.step()

        myA.data = stau(myA.data, tau_A*lr)


        mse_train.append(F.mse_loss(preds, target).item())
        nll_train.append(loss_nll.item())
        kl_train.append(loss_kl.item())

    #print(h_A.item())

    return np.mean(np.mean(kl_train)  + np.mean(nll_train)), np.mean(nll_train), np.mean(mse_train), origin_A, optimizer, lr



In [10]:
n = 30 # The number of samples of data.
d = 5 # The number of variables in data.
x_dims = 1 # The number of input dimensions: default 1.
z_dims = d # The number of latent variable dimensions: default the same as variable size.
epochs = 200 # Number of epochs to train.
batch_size = 10 # Number of samples per batch. note: should be divisible by sample size, otherwise throw an error.
n_var=5

n_times=30 #no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list=np.zeros((n_times,time_stamp,n_var,n_var))
B_list=np.zeros((n_times,d, d))
FDR_total=[]
TPR_total=[]
SHD_total=[]
time_list=[]
for replicate in range(n_times):
  seed=seed_list[replicate]
  print(seed)
  X_all=simulate_lsem_dynamic(W_all,Z_graph,30,10, 'Binary',noise_scale=0.1).reshape(330,5,1) #create data
  average_list=np.zeros((time_stamp,d, d))
  FDR_list_piece=[]
  TPR_list_piece=[]
  SHD_list_piece=[]
  base_DAG=np.zeros((5, 5))
  ####estimate at each time_stamp####
  timestart=time.time()
  for j in range(time_stamp):
  # ----------- Configurations:
      k_max_iter = int(1e2) # The max iteration number for searching parameters.
      original_lr = 3e-3  # Initial learning rate.
      encoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      decoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      temp = 0.5 # Temperature for Gumbel softmax.
      factor = True # Factor graph model.
      encoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      decoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      tau_A = 0. # Coefficient for L-1 norm of matrix B.
      lambda1 = 0. # Coefficient for DAG constraint h1(B).
      c_B = 1 # Coefficient for absolute value h1(B).
      h1_tol = 1e-8 # The tolerance of error of h1(B) to zero.
      lr_decay = 200 # After how many epochs to decay LR by a factor of gamma. 
      gamma = 1.0 # LR decay factor. 
      ######################


      X=X_all[(j*30):(j*30+30),:]


      np.random.seed(seed)
      random.seed(seed)
      torch.manual_seed(seed)
      feat_train = torch.FloatTensor(X)
      feat_valid = torch.FloatTensor(X)
      feat_test = torch.FloatTensor(X)

      # Reconstruct itself
      train_data = TensorDataset(feat_train, feat_train)
      valid_data = TensorDataset(feat_valid, feat_train)
      test_data = TensorDataset(feat_test, feat_train)

      train_loader = DataLoader(train_data, batch_size = batch_size)
      valid_loader = DataLoader(valid_data, batch_size = batch_size)
      test_loader = DataLoader(test_data, batch_size = batch_size)

      # ----------- Load modules:
      off_diag = np.ones([d, d]) - np.eye(d) # Generate off-diagonal interaction graph
      rel_rec = np.array(encode_onehot(np.where(off_diag)[1]), dtype = np.float64)
      rel_send = np.array(encode_onehot(np.where(off_diag)[0]), dtype = np.float64)
      rel_rec = torch.DoubleTensor(rel_rec)
      rel_send = torch.DoubleTensor(rel_send)
      adj_A = np.zeros((d, d)) # Add adjacency matrix

      encoder = MLPEncoder(d * x_dims, x_dims, encoder_hidden,
                              int(z_dims), adj_A,
                              batch_size = batch_size,
                              do_prob = encoder_dropout, factor = factor).double()
      decoder = MLPDecoder(d * x_dims,
                              z_dims, x_dims, encoder,
                              data_variable_size = d,
                              batch_size = batch_size,
                              n_hid=decoder_hidden,
                              do_prob=decoder_dropout).double()

      # ----------- Set up optimizer:
      optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr = original_lr)
      scheduler = lr_scheduler.StepLR(optimizer, step_size = lr_decay,
                                      gamma = gamma)

      rel_rec = Variable(rel_rec)
      rel_send = Variable(rel_send)

      # ----------- Main:
      best_ELBO_loss = np.inf
      best_NLL_loss = np.inf
      best_MSE_loss = np.inf
      h1_B_new = torch.tensor(1.)
      h2_B_new = 1
      h1_B_old = np.inf
      h2_B_old = np.inf
      lr = original_lr
    

      try:
          for step_k in range(k_max_iter):
              while c_B< 1e+20:
                  for epoch in range(epochs):
                      old_lr = lr 
                      ELBO_loss, NLL_loss, MSE_loss, origin_B, optimizer, lr = train(epoch, lambda1, c_B, optimizer, old_lr)

                      if ELBO_loss < best_ELBO_loss:
                          best_ELBO_loss = ELBO_loss

                      if NLL_loss < best_NLL_loss:
                          best_NLL_loss = NLL_loss

                      if MSE_loss < best_MSE_loss:
                          best_MSE_loss = MSE_loss

                  if ELBO_loss > 2 * best_ELBO_loss:
                      break

                  # Update parameters
                  B_new = origin_B.data.clone()
                  h1_B_new =  _h_A(B_new,d)
                  if h1_B_new.item() > 0.25 * h1_B_old:
                      c_B *= 10
                  else:
                      break

              # Update parameters    
              h1_B_old = h1_B_new.item()
              lambda1 += c_B * h1_B_new.item()

              if h1_B_new.item() <= h1_tol:
                  break

      except KeyboardInterrupt:
          print('KeyboardInterrupt')

      predB = np.matrix(origin_B.data.clone().numpy())
      print('Best ELBO Loss :', best_ELBO_loss)
      print('Best NLL Loss :', best_NLL_loss)
      print('Best MSE Loss :', best_MSE_loss)
      #calculate_effect(predB)
      print(j)
      average_list[j,:,:]=predB



  average_coef_list[replicate,:,:,:]=average_list #average coef save to matrix
  np.save("cos_10_30_DAGGNN_lag",average_coef_list)
#   timeend=time.time()
#   time_list.append(timeend-timestart)
  #print(timeend-timestart)
  #####write at every epoch
  #df.to_csv("cos_rep10.csv")
  print(replicate)

913812


D:\Anaconda\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Best ELBO Loss : 0.0008061077547894033
Best NLL Loss : 8.072632614576263e-07
Best MSE Loss : 3.229053045830505e-07
0
Best ELBO Loss : 0.0011623176728201097
Best NLL Loss : 1.4281851884240224e-06
Best MSE Loss : 5.712740753696089e-07
1
Best ELBO Loss : 0.0012245781211142036
Best NLL Loss : 1.413334749139061e-06
Best MSE Loss : 5.653338996556244e-07
2
Best ELBO Loss : 0.0015592984870046
Best NLL Loss : 2.504796663482094e-06
Best MSE Loss : 1.0019186653928375e-06
3
Best ELBO Loss : 0.002021875796399089
Best NLL Loss : 2.2973898957959417e-06
Best MSE Loss : 9.189559583183768e-07
4
Best ELBO Loss : 0.002676206001566601
Best NLL Loss : 4.041692016203852e-05
Best MSE Loss : 1.616676806481541e-05
5
Best ELBO Loss : 0.003567298174093924
Best NLL Loss : 4.813727589476105e-05
Best MSE Loss : 1.925491035790442e-05
6
Best ELBO Loss : 0.0034216217730273974
Best NLL Loss : 2.7938287417382085e-06
Best MSE Loss : 1.1175314966952836e-06
7
Best ELBO Loss : 0.0033844190534963762
Best NLL Loss : 7.51953321

Best ELBO Loss : 0.0013654468856226852
Best NLL Loss : 3.1820684239401673e-05
Best MSE Loss : 1.2728273695760672e-05
0
Best ELBO Loss : 0.0016509682286181759
Best NLL Loss : 4.726291070251566e-06
Best MSE Loss : 1.8905164281006258e-06
1
Best ELBO Loss : 0.0018913988756496041
Best NLL Loss : 2.4643681472153393e-06
Best MSE Loss : 9.857472588861356e-07
2
Best ELBO Loss : 0.001708539558597654
Best NLL Loss : 2.533309569656081e-06
Best MSE Loss : 1.0133238278624323e-06
3
Best ELBO Loss : 0.001871336332881541
Best NLL Loss : 1.7612785226394425e-06
Best MSE Loss : 7.045114090557771e-07
4
Best ELBO Loss : 0.0023122388349361466
Best NLL Loss : 3.963149462503757e-06
Best MSE Loss : 1.5852597850015029e-06
5
Best ELBO Loss : 0.003006422076299626
Best NLL Loss : 4.78148312198538e-06
Best MSE Loss : 1.912593248794152e-06
6
Best ELBO Loss : 0.0041214706483711045
Best NLL Loss : 4.360089459760662e-06
Best MSE Loss : 1.7440357839042649e-06
7
Best ELBO Loss : 0.003915606311423931
Best NLL Loss : 2.4739

Best ELBO Loss : 0.0011701648612113043
Best NLL Loss : 1.2991225527646512e-06
Best MSE Loss : 5.196490211058605e-07
0
Best ELBO Loss : 0.0021611253238672163
Best NLL Loss : 3.5961488059411854e-06
Best MSE Loss : 1.4384595223764742e-06
1
Best ELBO Loss : 0.002424495177706229
Best NLL Loss : 4.129037350941601e-06
Best MSE Loss : 1.65161494037664e-06
2
Best ELBO Loss : 0.0030240385244388653
Best NLL Loss : 8.439651120263805e-06
Best MSE Loss : 3.3758604481055228e-06
3
Best ELBO Loss : 0.0028015101618954104
Best NLL Loss : 2.957886973200865e-06
Best MSE Loss : 1.1831547892803458e-06
4
Best ELBO Loss : 0.003710003887560469
Best NLL Loss : 7.88654912938187e-06
Best MSE Loss : 3.1546196517527477e-06
5
Best ELBO Loss : 0.002841046653615495
Best NLL Loss : 3.987875086175913e-06
Best MSE Loss : 1.5951500344703654e-06
6
Best ELBO Loss : 0.002186338574316911
Best NLL Loss : 4.261528422541487e-06
Best MSE Loss : 1.7046113690165947e-06
7
Best ELBO Loss : 0.0032254611290208277
Best NLL Loss : 7.58083

Best ELBO Loss : 0.0010877894298608679
Best NLL Loss : 7.37199799820976e-06
Best MSE Loss : 2.9487991992839036e-06
0
Best ELBO Loss : 0.001357803619743696
Best NLL Loss : 1.8579303607272663e-06
Best MSE Loss : 7.431721442909067e-07
1
Best ELBO Loss : 0.0014053802266007606
Best NLL Loss : 1.7215377298156518e-06
Best MSE Loss : 6.886150919262608e-07
2
Best ELBO Loss : 0.0015003073797442109
Best NLL Loss : 1.4344591460132396e-06
Best MSE Loss : 5.737836584052958e-07
3
Best ELBO Loss : 0.0020258688336808692
Best NLL Loss : 2.375872378394564e-06
Best MSE Loss : 9.503489513578255e-07
4
Best ELBO Loss : 0.002057956745309163
Best NLL Loss : 2.0833352386057e-06
Best MSE Loss : 8.333340954422799e-07
5
Best ELBO Loss : 0.0022510256329444332
Best NLL Loss : 3.73627143690693e-06
Best MSE Loss : 1.4945085747627718e-06
6
Best ELBO Loss : 0.002383940210437804
Best NLL Loss : 4.609870309598561e-06
Best MSE Loss : 1.8439481238394245e-06
7
Best ELBO Loss : 0.0027959935316484333
Best NLL Loss : 3.69630779

Best ELBO Loss : 0.0018107679326537323
Best NLL Loss : 3.969248310470794e-06
Best MSE Loss : 1.5876993241883178e-06
0
Best ELBO Loss : 0.0028277932241369532
Best NLL Loss : 6.616860516715846e-06
Best MSE Loss : 2.6467442066863386e-06
1
Best ELBO Loss : 0.0028595247070894053
Best NLL Loss : 7.284467942258659e-06
Best MSE Loss : 2.9137871769034636e-06
2
Best ELBO Loss : 0.0032938418040141366
Best NLL Loss : 7.307413350654961e-06
Best MSE Loss : 2.922965340261985e-06
3
Best ELBO Loss : 0.0034986130266166132
Best NLL Loss : 8.119723362743487e-06
Best MSE Loss : 3.2478893450973945e-06
4
Best ELBO Loss : 0.004493537357890887
Best NLL Loss : 1.2140634940163769e-05
Best MSE Loss : 4.856253976065507e-06
5
Best ELBO Loss : 0.004679285043961312
Best NLL Loss : 1.3728248850133035e-05
Best MSE Loss : 5.491299540053215e-06
6
Best ELBO Loss : 0.005906140879472833
Best NLL Loss : 1.4785024362118608e-05
Best MSE Loss : 5.914009744847443e-06
7
Best ELBO Loss : 0.004253403341772251
Best NLL Loss : 1.4789